# Dataset Cleaning
*(This notebook was inspired by Anton T. Ruberts' Dataset Cleaning notebook.)*
The dataset came from the [Coffee Shop Sales](https://mavenanalytics.io/data-playground) from MavenAnalytics.

The main objectives of this notebook are:
- Observe the contents of the dataset,
- handle missing, duplicate, incorrect, or outlier values, and
- export the cleaned data.

In [4]:
import pandas as pd

## Loading the dataset

In [13]:
data = pd.read_csv('../data/coffee.csv')
data.head(-10)

transaction_id transaction_date transaction_time  store_id  \
0                    1         1/1/2023          7:06:11         5   
1                    2         1/1/2023          7:08:56         5   
2                    3         1/1/2023          7:14:04         5   
3                    4         1/1/2023          7:20:24         5   
4                    5         1/1/2023          7:22:41         5   
...                ...              ...              ...       ...   
149101          149442       30-06-2023         19:46:28         8   
149102          149443       30-06-2023         19:46:43         3   
149103          149444       30-06-2023         19:51:59         3   
149104          149445       30-06-2023         19:52:28         3   
149105          149446       30-06-2023         19:53:46         3   

         store_location  product_id  transaction_qty  unit_price  Total_Bill  \
0       Lower Manhattan          32                2        3.00         6.0   
1       Lower Manhattan          57                2        3.10         6.2   
2       Lower Manhattan          59                2        4.50         9.0   
3       Lower Manhattan          22                1        2.00         2.0   
4       Lower Manhattan          57                2        3.10         6.2   
...                 ...         ...              ...         ...         ...   
149101   Hell's Kitchen          75                1        3.50         3.5   
149102          Astoria          50                2        2.50         5.0   
149103          Astoria          26                2        3.00         6.0   
149104          Astoria          34                2        2.45         4.9   
149105          Astoria          24                1        3.00         3.0   

          product_category           product_type            product_detail  \
0                   Coffee  Gourmet brewed coffee                  Ethiopia   
1                      Tea        Brewed Chai tea     Spicy Eye Opener Chai   
2       Drinking Chocolate          Hot chocolate            Dark chocolate   
3                   Coffee            Drip coffee  Our Old Time Diner Blend   
4                      Tea        Brewed Chai tea     Spicy Eye Opener Chai   
...                    ...                    ...                       ...   
149101              Bakery                 Pastry                 Croissant   
149102                 Tea       Brewed Black tea                 Earl Grey   
149103              Coffee  Organic brewed coffee                 Brazilian   
149104              Coffee  Premium brewed coffee     Jamaican Coffee River   
149105              Coffee            Drip coffee  Our Old Time Diner Blend   

               Size Month Name Day Name  Hour  Month  Day of Week  
0           Regular    January   Sunday     7      1            6  
1             Large    January   Sunday     7      1            6  
2             Large    January   Sunday     7      1            6  
3             Small    January   Sunday     7      1            6  
4             Large    January   Sunday     7      1            6  
...             ...        ...      ...   ...    ...          ...  
149101  Not Defined       June   Friday    19      6            4  
149102      Regular       June   Friday    19      6            4  
149103      Regular       June   Friday    19      6            4  
149104        Small       June   Friday    19      6            4  
149105        Large       June   Friday    19      6            4  

[149106 rows x 18 columns]

## Initial exploration of the data

In [14]:
print("Dataset shape:", data.shape)
print("Dataset columns:", data.columns)

Dataset shape: (149116, 18)
Dataset columns: Index(['transaction_id', 'transaction_date', 'transaction_time', 'store_id',
       'store_location', 'product_id', 'transaction_qty', 'unit_price',
       'Total_Bill', 'product_category', 'product_type', 'product_detail',
       'Size', 'Month Name', 'Day Name', 'Hour', 'Month', 'Day of Week'],
      dtype='object')


In [15]:
data.describe()

transaction_id       store_id     product_id  transaction_qty  \
count   149116.000000  149116.000000  149116.000000    149116.000000   
mean     74737.371872       5.342063      47.918607         1.438276   
std      43153.600016       2.074241      17.930020         0.542509   
min          1.000000       3.000000       1.000000         1.000000   
25%      37335.750000       3.000000      33.000000         1.000000   
50%      74727.500000       5.000000      47.000000         1.000000   
75%     112094.250000       8.000000      60.000000         2.000000   
max     149456.000000       8.000000      87.000000         8.000000   

          unit_price     Total_Bill           Hour          Month  \
count  149116.000000  149116.000000  149116.000000  149116.000000   
mean        3.382219       4.686367      11.735790       3.988881   
std         2.658723       4.227099       3.764662       1.673091   
min         0.800000       0.800000       6.000000       1.000000   
25%         2.500000       3.000000       9.000000       3.000000   
50%         3.000000       3.750000      11.000000       4.000000   
75%         3.750000       6.000000      15.000000       5.000000   
max        45.000000     360.000000      20.000000       6.000000   

         Day of Week  
count  149116.000000  
mean        2.982336  
std         1.996650  
min         0.000000  
25%         1.000000  
50%         3.000000  
75%         5.000000  
max         6.000000

**Observations**
- The only quantitative columns in the dataset are: `transaction_qty`, `unit_price`, and `Total_Bill`.
- Among the abovementioned, all three columns have an outlier. A notable observation is that the maximum value in the `Total_Bill` column appears to be influenced by both the maximum values of`transaction_qty` and `unit_price` variables.

**Actions(s)**
- We can remove the `product_id` and `store_id` since `product_type` and `store_location` are much simpler entries. The same goes for droppable time columns `Month` and `Day of Week`, which have better counterparts in `Month Name` and `Day Name`.
- `transaction_id` is not relevant because there are no merges to be performed, thus can be dropped.


In [16]:
data.drop(['transaction_id', 'transaction_time', 'store_id', 'product_id', 'Month', 'Day of Week'], axis=1, inplace=True)

### Missing Data

In [17]:
data.isna().sum()

transaction_date    0
store_location      0
transaction_qty     0
unit_price          0
Total_Bill          0
product_category    0
product_type        0
product_detail      0
Size                0
Month Name          0
Day Name            0
Hour                0
dtype: int64

**Observations**
- Where do the missing data come from? Are there missing entries associated with a category/value of a column?

**Outcome(s)**
- Can the rows be dropped? Why?

## Data Pre-processing Pipeline

For this coffee dataset, we have a clean dataset such that we do not have to apply a pipeline. Nonetheless, I'm leaving this section here to serve as a template for future projects.

In [19]:
data['transaction_date'] = pd.to_datetime(data['transaction_date'], format="mixed", dayfirst=True)
data.to_csv('../data/coffee_cleaned.csv', index=False)  

In [20]:
data.head()

transaction_date   store_location  transaction_qty  unit_price  Total_Bill  \
0       2023-01-01  Lower Manhattan                2         3.0         6.0   
1       2023-01-01  Lower Manhattan                2         3.1         6.2   
2       2023-01-01  Lower Manhattan                2         4.5         9.0   
3       2023-01-01  Lower Manhattan                1         2.0         2.0   
4       2023-01-01  Lower Manhattan                2         3.1         6.2   

     product_category           product_type            product_detail  \
0              Coffee  Gourmet brewed coffee                  Ethiopia   
1                 Tea        Brewed Chai tea     Spicy Eye Opener Chai   
2  Drinking Chocolate          Hot chocolate            Dark chocolate   
3              Coffee            Drip coffee  Our Old Time Diner Blend   
4                 Tea        Brewed Chai tea     Spicy Eye Opener Chai   

      Size Month Name Day Name  Hour  
0  Regular    January   Sunday     7  
1    Large    January   Sunday     7  
2    Large    January   Sunday     7  
3    Small    January   Sunday     7  
4    Large    January   Sunday     7